### Purpose:

To get experience using CSVkit and the excel extension of Agate.

The data we will be using is a subset of the United States Defense Logistic Agency Law Enforcement Support Office’s (LESO) 1033 Program dataset, which describes how surplus military arms have been distributed to local police forces. This data was widely cited in the aftermath of the Ferguson, Missouri protests. The particular data we are using comes from an NPR report analyzing the data.





#### Load dependencies and globals

In [1]:
import csvkit
import agate
import agateexcel
from agatehacks import to_one_place, Percent
import os

THIS_DIR = os.getcwd()


## CSVKIT
### Read in the excel file and make CSV

In [2]:
# these 3 commands  work from the command line (from within the repo directory and an activated ve)
# but not here inside the notebook.
#Since csvkit was made to run from the command line there are things you have to add to make it run inside python.
#because  agate replaces csvkit I decided to not worry about running csvkit within notebook.

# in2csv ne_1033_data.xlsx  > LESO.csv
# csvcut -n LESO.csv  see the columns
# csvcut LESO.CSV |csvstat

## Move to Agate

### read the xlsx file and write out a csv

In [3]:
#the documentation is wrong.   I need to pip install agate-excel not agateexcel
agateexcel.patch()
table = agate.Table.from_xls('ne_1033_data.xlsx')
print(table)
table.to_csv ('xxx.csv')

|-------------------------------+---------------|
|  column_names                 | column_types  |
|-------------------------------+---------------|
|  state                        | Text          |
|  county                       | Text          |
|  fips                         | Number        |
|  nsn                          | Text          |
|  item_name                    | Text          |
|  quantity                     | Number        |
|  ui                           | Text          |
|  acquisition_cost             | Number        |
|  total_cost                   | Number        |
|  ship_date                    | Date          |
|  federal_supply_category      | Number        |
|  federal_supply_category_name | Text          |
|  federal_supply_class         | Number        |
|  federal_supply_class_name    | Text          |
|-------------------------------+---------------|



### read the CSV file  made with  CSVKit(command line) into Agate 
### crosstabs the old way using aggregate and compute methods

In [4]:
LESO = agate.Table.from_csv('LESO.csv')
n = len(LESO.rows)
print "%s rows in table" % n
#x= type(LESO)
#print 'LESO is of type %s' %x
#print dir(LESO)



#what aare the columns in LESO
print(LESO)
LESO.print_table(max_rows = 15)
item_freq = LESO.group_by("item_name")
item_freq = item_freq.aggregate([
    ('count', agate.Count())
])

item_freq = item_freq.compute([
    ('percent', Percent('count')),
])
item_freq = item_freq.order_by('count', reverse=True)

item_freq.print_table(max_rows=5)



1036 rows in table
|-------------------------------+---------------|
|  column_names                 | column_types  |
|-------------------------------+---------------|
|  state                        | Text          |
|  county                       | Text          |
|  fips                         | Number        |
|  nsn                          | Text          |
|  item_name                    | Text          |
|  quantity                     | Number        |
|  ui                           | Text          |
|  acquisition_cost             | Number        |
|  total_cost                   | Number        |
|  ship_date                    | Date          |
|  federal_supply_category      | Number        |
|  federal_supply_category_name | Text          |
|  federal_supply_class         | Number        |
|  federal_supply_class_name    | Text          |
|-------------------------------+---------------|

|--------+---------+--------+------------------+----------------------+---------

### crosstabs using pivot tables by emulating excel
####  from documentation at: http://agate.readthedocs.org/en/1.3.0/cookbook/excel.html#simple-formulas

In [5]:
x = type(LESO)
print 'LESO is of type %s' %x
print "*************WHY THE DIFFERENCE***********************************"
print ''
print 'This  is dir of agate.table '
print dir('agate.table.Table')
print ''
print 'This is dir of LESO'
print dir(LESO)
print ""
a = LESO.pivot('item_name')
print 'this is a count - no percent'
a.print_table(max_rows= 10)
print ''




LESO is of type <class 'agate.table.Table'>
*************WHY THE DIFFERENCE***********************************

This  is dir of agate.table 
['__add__', '__class__', '__contains__', '__delattr__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getslice__', '__gt__', '__hash__', '__init__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_formatter_field_name_split', '_formatter_parser', 'capitalize', 'center', 'count', 'decode', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'index', 'isalnum', 'isalpha', 'isdigit', 'islower', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']

#### this is a count - 2 way crosstabe - no percent'

In [6]:


print 'this is a count - 2 way crosstabe - no percent'
a2 = LESO.where(lambda row: row['state'] != None or  row['item_name'] != None)
a3 = a2.pivot('county', 'item_name')
a3.print_table(max_columns= 5) 


this is a count - 2 way crosstabe - no percent
|-------------+-----------------------+-----------------------+-------------+------------------------------+------|
|  county     | RIFLE,7.62 MILLIMETER | RIFLE,5.56 MILLIMETER | TRUCK,CARGO | PISTOL,CALIBER .45,AUTOMATIC | ...  |
|-------------+-----------------------+-----------------------+-------------+------------------------------+------|
|  ADAMS      |                     6 |                     0 |           0 |                            0 | ...  |
|  BUFFALO    |                     0 |                     8 |           0 |                            0 | ...  |
|  BURT       |                     1 |                     9 |           1 |                            0 | ...  |
|  BUTLER     |                     1 |                     0 |           0 |                            0 | ...  |
|  CASS       |                     0 |                    37 |           0 |                            0 | ...  |
|  CHASE      |          

#### this is a count - 2 way crosstabe - no percent'

In [7]:
print 'this is a count - 2 way crosstabe - no percent'
a1 = LESO.pivot('state', 'county')
a1.print_table(max_columns= 10)


this is a count - 2 way crosstabe - no percent
|--------+-------+---------+------+--------+------+-------+--------+--------+-------+------|
|  state | ADAMS | BUFFALO | BURT | BUTLER | CASS | CHASE | COLFAX | DAKOTA | DAWES | ...  |
|--------+-------+---------+------+--------+------+-------+--------+--------+-------+------|
|  NE    |     6 |       8 |   11 |      1 |   37 |     4 |      8 |     42 |     7 | ...  |
|--------+-------+---------+------+--------+------+-------+--------+--------+-------+------|


#### pivot tables as summaries

In [8]:


jobs = LESO.group_by('county')
summary = jobs.aggregate([
    ('record_count', agate.Count()),
    ('acquisition_cost', agate.Sum('acquisition_cost')),
    ('quantity', agate.Sum('quantity'))
])
summary.print_table()


|-------------+--------------+------------------+-----------|
|  county     | record_count | acquisition_cost | quantity  |
|-------------+--------------+------------------+-----------|
|  ADAMS      |            6 |           828.00 |        6  |
|  BUFFALO    |            8 |         3,992.00 |        8  |
|  BURT       |           11 |         5,099.00 |       11  |
|  BUTLER     |            1 |           138.00 |        1  |
|  CASS       |           37 |        16,189.00 |       37  |
|  CHASE      |            4 |           480.00 |        4  |
|  COLFAX     |            8 |           978.00 |        8  |
|  DAKOTA     |           42 |        53,262.46 |      101  |
|  DAWES      |            7 |           840.00 |        7  |
|  DAWSON     |           12 |         1,656.00 |       12  |
|  DIXON      |            5 |         2,495.00 |        5  |
|  DODGE      |            7 |           966.00 |        7  |
|  DOUGLAS    |          760 |     4,809,972.89 |    2,422  |
|  FILLM

### crosstabs using pivot tables using Transform. 
- NOT SURE I LIKE THIS BETTER
- http://agate.readthedocs.org/en/1.3.0/cookbook/transform.html#pivot-to-percent-of-total

In [ ]:
 ### table can be pivoted to count the number occurences of values in a column:

In [9]:
transformed1 = LESO.pivot('county')
transformed1.print_table(max_rows=10)


|----------+--------|
|  county  | Count  |
|----------+--------|
|  ADAMS   |     6  |
|  BUFFALO |     8  |
|  BURT    |    11  |
|  BUTLER  |     1  |
|  CASS    |    37  |
|  CHASE   |     4  |
|  COLFAX  |     8  |
|  DAKOTA  |    42  |
|  DAWES   |     7  |
|  DAWSON  |    12  |
|  ...     |   ...  |
|----------+--------|


In [10]:
###Pivot by multiple columns - 2 way crosstab

In [11]:
a2 = LESO.where(lambda row: row['state'] != None or  row['item_name'] != None)
transformed2 = a2.pivot(['county', 'item_name'])
transformed2.print_table(max_rows =20, max_columns = 5)


|----------+----------------------+--------|
|  county  | item_name            | Count  |
|----------+----------------------+--------|
|  ADAMS   | RIFLE,7.62 MILLIM... |     6  |
|  BUFFALO | RIFLE,5.56 MILLIM... |     8  |
|  BURT    | RIFLE,5.56 MILLIM... |     9  |
|  BURT    | RIFLE,7.62 MILLIM... |     1  |
|  BURT    | TRUCK,CARGO          |     1  |
|  BUTLER  | RIFLE,7.62 MILLIM... |     1  |
|  CASS    | RIFLE,5.56 MILLIM... |    37  |
|  CHASE   | RIFLE,5.56 MILLIM... |     4  |
|  COLFAX  | RIFLE,7.62 MILLIM... |     1  |
|  COLFAX  | RIFLE,5.56 MILLIM... |     7  |
|  DAKOTA  | RIFLE,5.56 MILLIM... |     2  |
|  DAKOTA  | PISTOL,CALIBER .4... |     5  |
|  DAKOTA  | REVOLVER,CALIBER ... |     5  |
|  DAKOTA  | ROBOT,EXPLOSIVE O... |     1  |
|  DAKOTA  | TOOL KIT             |     3  |
|  DAKOTA  | ILLUMINATOR,INFRARED |    21  |
|  DAKOTA  | PLATOON EARLY WARNI  |     3  |
|  DAKOTA  | SPECTACLES SET,BALL  |     2  |
|  DAWES   | RIFLE,5.56 MILLIM... |     7  |
|  DAWSON 

### Pivot to sum
 The default pivot aggregation is Count but you can also supply other operations. For example, to aggregate
 each group by Sum of their total costs:
 

In [12]:
#example:
# Burt county got 1 truck,cargo(see above) and it cost $3123.
# Burt county got 9 RIFLE,5.56 MILLIMeter at different prices  but the total was 1838 (verified) 
print 'total cost of items by county'
transformed3 = a2.pivot('county', 'item_name', aggregation=agate.Sum('total_cost'))
transformed3.print_table(max_rows =20, max_columns = 5)
#transformed3.to_csv ('transformed3.csv')

total cost of items by county
|------------+-----------------------+-----------------------+-------------+------------------------------+------|
|  county    | RIFLE,7.62 MILLIMETER | RIFLE,5.56 MILLIMETER | TRUCK,CARGO | PISTOL,CALIBER .45,AUTOMATIC | ...  |
|------------+-----------------------+-----------------------+-------------+------------------------------+------|
|  ADAMS     |                   828 |                     0 |           0 |                         0.00 | ...  |
|  BUFFALO   |                     0 |                 3,992 |           0 |                         0.00 | ...  |
|  BURT      |                   138 |                 1,838 |       3,123 |                         0.00 | ...  |
|  BUTLER    |                   138 |                     0 |           0 |                         0.00 | ...  |
|  CASS      |                     0 |                16,189 |           0 |                         0.00 | ...  |
|  CHASE     |                     0 |            

### Pivot to percent of total ( the total of all cells, not just the total of a row)

In [29]:
#using the example in the documentation: http://agate.readthedocs.org/en/1.3.0/cookbook/transform.html#pivot-to-percent-of-total


print ' example from documentation'
agateexcel.patch()
ex = agate.Table.from_xls('pivot_to_percent_example.xlsx')
print 'table used for example:'
ex.print_table()
print ''
print 'count race'
ex2 = ex.pivot('race')
ex2.print_table()
print''
print 'race by gender- sum on age'
print 'total of all ages is 135'
ex3 = ex.pivot('race', 'gender', aggregation=agate.Sum('age'))
ex3.print_table()
print''
print 'percent cell is of total ( pivot to percent of total)'
ex4 = ex.pivot('race', 'gender', aggregation=agate.Sum('age'), computation=agate.Percent('Sum'))
ex4.print_table()
print''
print 'percent cell rounded from Ben and requires agatehacks to be imported'
ex5 = ex4.compute([
   ('female_percent_of_total', agate.Formula(
        agate.Number(),
        lambda row: to_one_place((row['female'])
    ))),
         ('male_percent_of_total', agate.Formula(
         agate.Number(),
         lambda row: to_one_place((row['male'])
     ))),

])
ex5.print_table()

print '***** end of example ************************'




print ""
print ""
print 'back to the LESO data'
transformed4 =  a2.pivot('county', 'item_name', aggregation=agate.Sum('total_cost'),computation=agate.Percent('Sum'))
#transformed4.print_table(max_rows =20, max_columns = 5)
print ""
print ""
transformed5 = transformed4.compute([
   ('pct_RIFLE,7.62_of_total', agate.Formula(
        agate.Number(),
        lambda row: to_one_place((row['RIFLE,7.62 MILLIMETER'])
    ))),
         ('pct_RIFLE,5.56_of_total', agate.Formula(
         agate.Number(),
         lambda row: to_one_place((row['RIFLE,5.56 MILLIMETER'])
     ))),

])


print""
print ' here is another way to round  using from decimal import Decimal'

from decimal import Decimal

number_type = agate.Number()

def round_price(row):
    return row['TRUCK,CARGO'].quantize(Decimal('0.01'))

transformed5 = transformed5.compute([
    ('pct_TRUCK,CARGO_of_total', agate.Formula(number_type, round_price))
])

#so I can see my data
transformed5.to_csv ('transformed5.csv')
transformed6 = transformed5.select([
    'county',
    'RIFLE,5.56 MILLIMETER',
    'pct_RIFLE,5.56_of_total',
    'RIFLE,7.62 MILLIMETER',
    'pct_RIFLE,7.62_of_total',
    'TRUCK,CARGO',
    'pct_TRUCK,CARGO_of_total' ,   
]).order_by('county')
transformed6.to_csv ('transformed6.csv')




 example from documentation
table used for example:
|--------+--------+--------+------|
|  name  | race   | gender | age  |
|--------+--------+--------+------|
|  joe   | white  | female |  20  |
|  Jane  | asian  | male   |  20  |
|  Jill  | black  | female |  20  |
|  Jim   | latino | male   |  25  |
|  Julia | black  | female |  25  |
|  Joan  | asian  | female |  25  |
|--------+--------+--------+------|

count race
|---------+--------|
|  race   | Count  |
|---------+--------|
|  white  |     1  |
|  asian  |     2  |
|  black  |     2  |
|  latino |     1  |
|---------+--------|

race by gender- sum on age
total of all ages is 135
|---------+--------+-------|
|  race   | female | male  |
|---------+--------+-------|
|  white  |     20 |    0  |
|  asian  |     25 |   20  |
|  black  |     45 |    0  |
|  latino |      0 |   25  |
|---------+--------+-------|

percent cell is of total ( pivot to percent of total)
|---------+----------------------+-----------------------|
|  race  